In [5]:
import pandas as pd 
import numpy as np
df = pd.read_csv('../../governance - Metadata.csv')
df.head()


,Country Name,Country Code,Year,Control of Corruption Score,Government Effectiveness Score,Political Stability Score,Regulatory Quality Score,Rule of Law Score,Free Speech Score
0,Afghanistan,AFG,2016,-1.526172,-1.226401,-2.671054,-1.327688,-1.498478,-1.037838
1,Afghanistan,AFG,2017,-1.515626,-1.325545,-2.801084,-1.336691,-1.562423,-0.9908541
2,Afghanistan,AFG,2018,-1.487624,-1.453096,-2.763864,-1.133277,-1.664038,-1.011603
3,Afghanistan,AFG,2019,-1.400733,-1.463065,-2.655531,-1.120690,-1.713714,-1.006792
4,Afghanistan,AFG,2020,-1.475405,-1.523115,-2.729808,-1.378587,-1.807697,-1.083459


In [6]:
df.dtypes

Country Name                       object
Country Code                       object
Year                                int64
Control of Corruption Score       float64
Government Effectiveness Score    float64
Political Stability Score         float64
Regulatory Quality Score          float64
Rule of Law Score                 float64
Free Speech Score                  object
dtype: object

In [7]:
df=df.replace({'..':np.NaN})#replace all cells with .. with Nan so that python can evaluate it as missing value
df.isnull().sum()
df.fillna(value=np.NaN,inplace=True)

In [8]:
#create a new column named Age to represent the year of measurement, not necessary but just for conveninience.
dic = {2016:0,2017:1,2018:2,2019:3,2020:4}
df['Age']=df['Year'].map(dic)
df

,Country Name,Country Code,Year,Control of Corruption Score,Government Effectiveness Score,Political Stability Score,Regulatory Quality Score,Rule of Law Score,Free Speech Score,Age
0,Afghanistan,AFG,2016,-1.526172,-1.226401,-2.671054,-1.327688,-1.498478,-1.037838,0
1,Afghanistan,AFG,2017,-1.515626,-1.325545,-2.801084,-1.336691,-1.562423,-0.9908541,1
2,Afghanistan,AFG,2018,-1.487624,-1.453096,-2.763864,-1.133277,-1.664038,-1.011603,2
3,Afghanistan,AFG,2019,-1.400733,-1.463065,-2.655531,-1.120690,-1.713714,-1.006792,3
4,Afghanistan,AFG,2020,-1.475405,-1.523115,-2.729808,-1.378587,-1.807697,-1.083459,4
...,...,...,...,...,...,...,...,...,...,...
1065,Zimbabwe,ZWE,2016,-1.259222,-1.158399,-0.619938,-1.720110,-1.368501,-1.177471,0
1066,Zimbabwe,ZWE,2017,-1.271278,-1.182262,-0.708769,-1.561047,-1.373646,-1.195013,1
1067,Zimbabwe,ZWE,2018,-1.230871,-1.198210,-0.722774,-1.504843,-1.272543,-1.140979,2
1068,Zimbabwe,ZWE,2019,-1.236825,-1.204820,-0.959749,-1.462590,-1.257262,-1.159892,3


In [9]:
df['Free Speech Score']=df['Free Speech Score'].astype(float)# data type conversion so for future operation
df['Year']=df['Year'].astype(int)

In [10]:
df.dtypes

Country Name                       object
Country Code                       object
Year                                int32
Control of Corruption Score       float64
Government Effectiveness Score    float64
Political Stability Score         float64
Regulatory Quality Score          float64
Rule of Law Score                 float64
Free Speech Score                 float64
Age                                 int64
dtype: object

## Missing value
The big idea behind this is to fill in missing value usign a regression line (y=ax+b). 
This is possible because of the data is a time serie data. This can work if  at least two values are known for the serie. in case one values is known, 
that one value is used to fill in the other value. if no value is known, then it default to zero.


In [11]:
def fill_missing_values(group):
    value_columns = group.select_dtypes(include='number').columns

    for value_col in value_columns:
        values = group[value_col]
        known_values_mask = values.notnull()
        if known_values_mask.sum() > 0:  # If there is at least one known value
            if known_values_mask.sum() > 1:  # If there are more than one known values
                sorted_group = group.sort_values('Age')
                x_known = group.loc[known_values_mask, 'Age']
                y_known = group.loc[known_values_mask, value_col]
                a = (y_known.iloc[-1] - y_known.iloc[0]) / (x_known.iloc[-1] - x_known.iloc[0])
                b = y_known.iloc[0] - (a * x_known.iloc[0])

                missing_mask = values.isnull()
                x_missing = group.loc[missing_mask, 'Age']
                group.loc[missing_mask, value_col] = a * x_missing + b
            else:  # If there is only one known value
                known_value = values[known_values_mask].iloc[0]
                group[value_col].fillna(known_value, inplace=True)


    return group

final = df.reset_index().groupby('Country Name',group_keys=True).apply(fill_missing_values).reset_index(drop=True)
final

,index,Country Name,Country Code,Year,Control of Corruption Score,Government Effectiveness Score,Political Stability Score,Regulatory Quality Score,Rule of Law Score,Free Speech Score,Age
0,0,Afghanistan,AFG,2016,-1.526172,-1.226401,-2.671054,-1.327688,-1.498478,-1.037838,0
1,1,Afghanistan,AFG,2017,-1.515626,-1.325545,-2.801084,-1.336691,-1.562423,-0.990854,1
2,2,Afghanistan,AFG,2018,-1.487624,-1.453096,-2.763864,-1.133277,-1.664038,-1.011603,2
3,3,Afghanistan,AFG,2019,-1.400733,-1.463065,-2.655531,-1.120690,-1.713714,-1.006792,3
4,4,Afghanistan,AFG,2020,-1.475405,-1.523115,-2.729808,-1.378587,-1.807697,-1.083459,4
...,...,...,...,...,...,...,...,...,...,...,...
1065,1065,Zimbabwe,ZWE,2016,-1.259222,-1.158399,-0.619938,-1.720110,-1.368501,-1.177471,0
1066,1066,Zimbabwe,ZWE,2017,-1.271278,-1.182262,-0.708769,-1.561047,-1.373646,-1.195013,1
1067,1067,Zimbabwe,ZWE,2018,-1.230871,-1.198210,-0.722774,-1.504843,-1.272543,-1.140979,2
1068,1068,Zimbabwe,ZWE,2019,-1.236825,-1.204820,-0.959749,-1.462590,-1.257262,-1.159892,3


In [19]:
final.fillna(value=0,inplace=True)# fill in Nan value with zero

In [20]:

final.drop('index',axis=1,inplace=True)# drop the index created while filling in the MV

In [21]:
# create an overall situation of the data
from ydata_profiling import  ProfileReport
prof = ProfileReport(final,title='eda',explorative=True,dark_mode=True)
prof.to_file('output_file_1.html')
# prof

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 107.15it/s]


In [23]:
final.to_csv('cleaned_governace_data.csv',sep=',')# convert the final cleaned data to a csv file.